### Standard setup

In [1]:
import sys
sys.path.append('/home/tp/gh/pulse-streamer/ni-streamer/py_api')
from nistreamer import NIStreamer

In [2]:
ni_strmr = NIStreamer()

ao_card = ni_strmr.add_ao_card('AODev', samp_rate=400e3)
do_card = ni_strmr.add_do_card('DODev', samp_rate=10e6)

ao_0 = ao_card.add_chan(chan_idx=0)
do_0 = do_card.add_chan(port_idx=0, line_idx=0)

In [3]:
ni_strmr.clear_edit_cache()

ao_0.sine(t=0, dur=1.0, amp=1.0, freq=1.234)
do_0.high(t=0.5, dur=1.0)

ni_strmr.compile()

1.5000000000000002

In [4]:
ni_strmr.starts_last = 'AODev'
ni_strmr.ref_clk_provider = ('AODev', 'TestPFI0')
ni_strmr.chunksize_ms = 123

# Stream controls

The most basic way of running the stream is to use the built-in `run` method:

In [5]:
ni_strmr.run(nreps=2)

For more advanced applications one can use the context manager API.
The simplest example is shown below - it is the equivalent of the above `run()` example (this is literally how `run` is implemented under the hood):

In [6]:
with ni_strmr.init_stream() as stream:
    stream.launch(nreps=2)
    stream.wait_until_finished()

Breakdown:

* `with ni_strmr.init_stream() as stream` initializes the stream and assigns `StreamHandle` instance to `stream` target;

* `launch(nreps)` method commands to launch the run with `nreps` _in-stream_ repetitions. **Note** - this call is non-blocking, it returns immediately;

* `wait_until_finished()` blocks and returns only when full waveform generation is finished.  
  If you want to stop generation before completing all `nreps`, emit `KeyboardInterrupt` - it will break out from `wait_until_finished()`

* Stream is automatically closed and all resources are released when leaving the context (this applies to both clean return and return due to an exception)

<div class="alert alert-block alert-info"> 
    <b>NOTE</b> You should always make a `wait_until_finished` call every time after you called `launch`.
</div>

<div class="alert alert-block alert-info"> 
    <b>NOTE</b> Both `run` and context-based controls can be intrrupted with `KeyboardInterrupt` if you don't want to wait for all reps to finish. After the signal was emitted, the streamer will finish a complete rep or two before stopping, it will not stop mid-way through the waveform.
</div>

`StreamHandle` class has the following methods (more info in corresponding docstrings):

* `launch`

* `wait_until_finished`

* `reps_written_count` - returns the total number of in-stream reps _written_ already (not the same as _generated_ already, see docstring)

* `request_stop` (you don't need this method in most cases)

Reasons for using  `with` context API:

* Avoids unnecessary re-inits (which typically take extra ~50 ms). Resource allocation (connecting to cards and configuring hardware, allocating large sample arrays) is only done once when entering the context. All resources are then reused for subsequent relaunches as long as you don't leave `with`;

* Custom "progress" monitoring for large `nreps` in-stream loops.


See examples below

## Specific examples

A series of single-rep launches without re-init overhead.  
Use cases:
* Wait for start trigger for each run
* Run custom code before/after each run (e.g. send network commands to other instruments, save data)

In [7]:
nlaunches = 3

with ni_strmr.init_stream() as stream:
    for _ in range(nlaunches):
        # before-run custom code here
        stream.launch()  # if start trigger was configured, each launch will wait for a trig to start
        stream.wait_until_finished()
        # after-run custom code here

Minimal example of a custom "progress bar" to monitor runs with a large number `nreps` of in-stream repetitions. 

Note that we are using `wait_until_finished` method with a non-`None` argument - it blocks until eithr generation is finished or timeout elapses (see docstring). That way we can periodically poll `reps_written_count()`

In [9]:
nreps = 10

with ni_strmr.init_stream() as stream:
    stream.launch(nreps=nreps)
    
    while True:
        finished = stream.wait_until_finished(timeout=1)
        print(f'{stream.reps_written_count()} reps written out of {nreps}', end='\r')
        if finished:
            break